In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import torch


In [21]:
class Pod:
    def __init__(self, pod_id, xy_angle, state_sequence=None, robot_id=None, path=None, property_value=None):
        self.podId = pod_id
        self.xya = xy_angle  # (x, y, angle)
        self.stateSequence = state_sequence
        self.robotId = robot_id
        self.Path = path
        self.Property = property_value

        
    def method1(self, input_arg):
        return input_arg+self.Property

In [11]:
class Task:
    def __init__(self, pod_id=None, station_id=None, process_time=None, return_state=None, status=None):
        self.PodID = pod_id
        self.StationID = station_id
        self.ProcessTime = process_time
        self.ReturnState = return_state
        self.Status = status

In [23]:
class Robot:
    def __init__(self, robot_id=None,pod_id=None, xy_angle=[0,0,0], action_sequence, state_sequence=None, path=None, task_id=None):
        self.ID = robot_id
        self.PodID = pod_id
        self.xya = xy_angle  # (x, y, angle)
        self.ActionSequence = action_sequence
        self.StateSequence = state_sequence
        self.Path = path
        self.Task = task_id


SyntaxError: expected ':' (380718086.py, line 1)

In [24]:
class warehouse:
    def __init__(self, x=None)

SyntaxError: expected ':' (725335420.py, line 2)

In [26]:
class Warehouse:
    def __init__(self, inputArg1, inputArg2):
        self.xlength = None
        self.ylength = None
        self.robotNum = None
        self.podNum = None
        self.depotNum = None
        self.Property1 = inputArg1 + inputArg2
    
    def method1(self, inputArg):
        outputArg = self.Property1 + inputArg
        return outputArg


In [33]:
def forward(x,y,ylength):
    return [ylength+1-y,x]
def inverse(r,c,ylength):
    return [c,ylength+1-r]
def initialize(xlength,ylength,robotNum,podNum,depotNum):
    RobotStates = np.zeros((robotNum,3))
    #x,y
    PodStates = np.zeros((podNum,3))
    #x,y
    DepotStates = np.zeros((depotNum,3))
    #storage locations
    StorageOccupancy = np.zeros((ylength,xlength))
    xy2rc=forward(x,y,ylength)
    rc2xy=inverse(r,c,ylength)
    for i in range(1, 21):
        StorageOccupancy[1:6, 2 + 3*(i-1):3 + 3*(i-1)] = 1
        StorageOccupancy[7:12, 2 + 3*(i-1):3 + 3*(i-1)] = 1
        StorageOccupancy[13:18, 2 + 3*(i-1):3 + 3*(i-1)] = 1
        StorageOccupancy[19:24, 2 + 3*(i-1):3 + 3*(i-1)] = 1
    R,C=np.where(StorageOccupancy==1)
    for i in range(1,podNum):
        PodStates[i,1:2]=rc2xy(R[i,1],C[i,1]);
        PodStates[i,3]=0
    R,C=np.where(StorageOccupancy==0)
    sb=np.size(R,1)
    sb=np.random.permutation(sb)
    for i in range(1, robotNum):
        index = sb[i]
        RobotStates[i, 1:2] = rc2xy(R[index, 1], C[index, 1])
        RobotStates[i, 3] = np.random.randint(1, 4)
    
    DepotStates[:, 0] = [4, 11, 18, 25, 32, 39, 46, 53]
    DepotStates[:, 1] = np.ones(depotNum)
    DepotStates[:, 2] = np.zeros(depotNum)
    
    return RobotStates, PodStates, DepotStates, StorageOccupancy

In [1]:

def AStarSTDiff(MapMat, startState, goalState, constraints):
    height, width = MapMat.shape
    actions = 4  # actions: 1=forward,2=turn left,3=turn right,4=stay
    
    # pre-assign memory
    OPEN_COUNT = 1
    MAX_OPEN_SIZE = 200
    OPEN = np.zeros((MAX_OPEN_SIZE, 13))
    OPEN_CHECK = np.zeros((MAX_OPEN_SIZE, 1))
    
    CLOSED_COUNT = 1
    MAX_CLOSED_SIZE = 200
    CLOSED = np.zeros((MAX_CLOSED_SIZE, 13))
    
    # Initialize start node with FValue and open first node.
    #error
    dist = abs(startState[0][0] - goalState[0][0]) + abs(startState[0][1] - goalState[0][1])
    # Node definition: R,C,A,T,parentR,parentC,parentA,parentT,F,G,H values,energy,actions
    Root = np.zeros(13)
    Root[:3] = startState
    Root[8] = dist
    OPEN[0] = Root
    OPEN_CHECK[0] = 1
    
    while np.any(OPEN_CHECK == 1):
        # best-first search and tie-breaking policy
        uncheckedIndex = np.where(OPEN_CHECK == 1)[0]
        TEMPOPEN = OPEN[uncheckedIndex]
        minFScore = np.min(TEMPOPEN[:, 8])
        index = np.where(TEMPOPEN[:, 8] == minFScore)[0]
        energy = TEMPOPEN[index, 11]  # tie-breaking
        index2 = np.argmin(energy)
        currentNodeIndex = uncheckedIndex[index[index2]]
        currentNode = TEMPOPEN[index[index2]]
        
        if np.all(currentNode[:3] == goalState):
            break
        
        # Removing node from OpenList to ClosedList
        temp = np.where(np.all(OPEN == currentNode, axis=1))[0][0]
        OPEN_CHECK[temp] = 0
        CLOSED[CLOSED_COUNT] = currentNode
        CLOSED_COUNT += 1
        
        for i in range(1, actions + 1):
            newNode = np.zeros(13)
            # state machine for the next state
            currentRCA = currentNode[:3]
            newState = currentRCA.copy()
            
            if i == 1:
                if currentRCA[2] == 1:
                    newState[1] += 1
                elif currentRCA[2] == 2:
                    newState[0] -= 1
                elif currentRCA[2] == 3:
                    newState[1] -= 1
                elif currentRCA[2] == 4:
                    newState[0] += 1
            elif i == 2:
                if currentRCA[2] == 1:
                    newState[2] = 2
                elif currentRCA[2] == 2:
                    newState[2] = 3
                elif currentRCA[2] == 3:
                    newState[2] = 4
                elif currentRCA[2] == 4:
                    newState[2] = 1
            elif i == 3:
                if currentRCA[2] == 1:
                    newState[2] = 4
                elif currentRCA[2] == 2:
                    newState[2] = 1
                elif currentRCA[2] == 3:
                    newState[2] = 2
                elif currentRCA[2] == 4:
                    newState[2] = 3
            
            newNode[:3] = newState
            newNode[3] = currentNode[3] + 1
            
            # Ignore the out of borders and obstacles.
            if newNode[0] < 1 or newNode[0] > height or newNode[1] < 1 or newNode[1] > width or MapMat[int(newNode[0]), int(newNode[1])] == 1:
                continue
            
            # Ignore the nodes in CLOSED lists.
            if np.any(np.all(CLOSED[:, :4] == newNode[:4], axis=1)):
                continue
            
            # Ignore the nodes in constraints lists.
            if constraints.size != 0:
                temp = newNode[:3].copy()
                temp[2] = newNode[3]
                if np.any(np.all(constraints[:, :3] == temp, axis=1)):
                    continue
            
            # Discover a new node and update its tentative F,G,H values
            newNode[4:8] = currentNode[:4]
            dist = abs(newNode[0] - goalState[0]) + abs(newNode[1] - goalState[1])
            newNode[8:11] = [dist + currentNode[9] + 1, currentNode[9] + 1, dist]
            
            if i == 4:
                newNode[11] = currentNode[11]
            else:
                newNode[11] = currentNode[11] + 1
            
            flag, index = np.any(np.all(OPEN[:, :4] == newNode[:4], axis=1)), 0
            if flag:  # if this node is already in the OPEN list
                index = np.where(np.all(OPEN[:, :4] == newNode[:4], axis=1))[0][0]
                if newNode[8] > OPEN[index, 8]:
                    continue
                else:
                    OPEN[index] = newNode
            else:  # if this node is not in the OPEN list
                OPEN[OPEN_COUNT] = newNode
                OPEN_CHECK[OPEN_COUNT] = 1
                OPEN_COUNT += 1
    
    # backward reconstruct path
    k = 0
    OptimalPath = np.zeros((200, 4))
    while not np.all(Root == currentNode):
        OptimalPath[k] = currentNode[:4]
        parentIndex = np.where(np.all(CLOSED[:, :4] == currentNode[4:8], axis=1))[0][0]
        currentNode = CLOSED[parentIndex]
        k += 1
    
    OptimalPath = OptimalPath[:k]
    OptimalPath = np.vstack((OptimalPath, Root[:4]))
    OptimalPath = np.flipud(OptimalPath)
    
    return OptimalPath.astype(int)


In [4]:
def AStarSTDiffHWY(MapMat, Highway, startState, goalState, constraints):
    height, width = np.shape(MapMat)
    actions = 4  # actions: 1=forward,2=turn left,3=turn right,4=stay
    
    # pre-assign memory
    OPEN_COUNT = 1
    MAX_OPEN_SIZE = 200
    OPEN = np.zeros((MAX_OPEN_SIZE, 13))
    OPEN_CHECK = np.zeros((MAX_OPEN_SIZE, 1))
    
    CLOSED_COUNT = 1
    MAX_CLOSED_SIZE = 200
    CLOSED = np.zeros((MAX_CLOSED_SIZE, 13))
    
    # Initialize start node with FValue and open first node.
    dist = abs(startState[0][0] - goalState[0][0]) + abs(startState[0][1] - goalState[0][1])
    # Node definition: R,C,A,T,parentR,parentC,parentA,parentT,F,G,H values,energy,actions
    Root = np.zeros(13)
    Root[:4] = startState
    Root[8] = dist
    OPEN[0] = Root
    OPEN_CHECK[0] = 1
    
    while np.any(OPEN_CHECK == 1):
        # best-first search and tie-breaking policy
        uncheckedIndex = np.where(OPEN_CHECK == 1)[0]
        TEMPOPEN = OPEN[uncheckedIndex]
        minFScore = np.min(TEMPOPEN[:, 8])
        index = np.where(TEMPOPEN[:, 8] == minFScore)[0]
        energy = TEMPOPEN[index, 11]  # tie-breaking
        index2 = np.argmin(energy)
        currentNodeIndex = uncheckedIndex[index[index2]]
        currentNode = TEMPOPEN[index[index2]]
        
        if np.all(currentNode[:4] == goalState):
            break
        
        # Removing node from OpenList to ClosedList
        temp = np.where(np.all(OPEN == currentNode, axis=1))[0][0]
        OPEN_CHECK[temp] = 0
        CLOSED[CLOSED_COUNT] = currentNode
        CLOSED_COUNT += 1
        
        currentRCA = currentNode[:3]
        # find out highways
        mat1 = Highway[0]
        hdirection = mat1[int(currentRCA[0]), int(currentRCA[1])]
        mat2 = Highway[1]
        vdirection = mat2[int(currentRCA[0]), int(currentRCA[1])]
        
        feasible = []
        if hdirection != 0:
            if vdirection == 0:
                feasible = [[currentRCA[0], currentRCA[1], hdirection]]
            else:
                feasible = [[currentRCA[0], currentRCA[1], hdirection], [currentRCA[0], currentRCA[1], vdirection]]
        else:
            if vdirection != 0:
                feasible = [[currentRCA[0], currentRCA[1], vdirection]]
            else:
                r = currentRCA[0]
                c = currentRCA[1]
                feasible = [[r, c, 1], [r, c, 2], [r, c, 3], [r, c, 4]]
        
        if currentRCA[0] == goalState[0][0] or currentRCA[1] == goalState[0][1]:
            r = currentRCA[0]
            c = currentRCA[1]
            feasible = [[r, c, 1], [r, c, 2], [r, c, 3], [r, c, 4]]
        
        for i in range(1, actions + 1):
            newNode = np.zeros(13)
            # state machine for the next state
            newState = currentRCA.copy()
            
            if i == 1:
                if currentRCA[2] == 1:
                    newState[1] += 1
                elif currentRCA[2] == 2:
                    newState[0] -= 1
                elif currentRCA[2] == 3:
                    newState[1] -= 1
                elif currentRCA[2] == 4:
                    newState[0] += 1
            elif i == 2:  # turn left
                if currentRCA[2] == 1:
                    newState[2] = 2
                elif currentRCA[2] == 2:
                    newState[2] = 3
                elif currentRCA[2] == 3:
                    newState[2] = 4
                elif currentRCA[2] == 4:
                    newState[2] = 1
                
                if len(feasible) == 1:
                    if abs(feasible[0][2] - newState[2]) == 2 or feasible[0][2] == currentRCA[2]:
                        continue
                elif len(feasible) == 2:
                    if newState.tolist() not in feasible:
                        continue
            elif i == 3:
                if currentRCA[2] == 1:
                    newState[2] = 4
                elif currentRCA[2] == 2:
                    newState[2] = 1
                elif currentRCA[2] == 3:
                    newState[2] = 2
                elif currentRCA[2] == 4:
                    newState[2] = 3
                
                if len(feasible) == 1:
                    if abs(feasible[0][2] - newState[2]) == 2 or feasible[0][2] == currentRCA[2]:
                        continue
                elif len(feasible) == 2:
                    if newState.tolist() not in feasible:
                        continue
            elif i == 4:
                pass
            
            newNode[:3] = newState
            newNode[3] = currentNode[3] + 1
            
            # Ignore the out of borders and obstacles.
            if newNode[0] < 1 or newNode[0] > height or newNode[1] < 1 or newNode[1] > width or MapMat[int(newNode[0]), int(newNode[1])] == 1:
                continue
            
            # Ignore the nodes in CLOSED lists.
            if np.any(np.all(CLOSED[:, :4] == newNode[:4], axis=1)):
                continue
            
            # Ignore the nodes in constraints lists.
            if constraints.size != 0:
                temp = newNode[:3].copy()
                temp[2] = newNode[3]
                if np.any(np.all(constraints[:, :3] == temp, axis=1)):
                    continue
            
            # Discover a new node and update its tentative F,G,H values
            newNode[4:8] = currentNode[:4]
            dist = abs(newNode[0] - goalState[0][0]) + abs(newNode[1] - goalState[0][1])
            newNode[8:11] = [dist + currentNode[9] + 1, currentNode[9] + 1, dist]
            
            if i == 4:
                newNode[11] = currentNode[11]
            else:
                newNode[11] = currentNode[11] + 1
            
            flag, index = np.any(np.all(OPEN[:, :4] == newNode[:4], axis=1)), 0
            if flag:  # if this node is already in the OPEN list
                index = np.where(np.all(OPEN[:, :4] == newNode[:4], axis=1))[0][0]
                if newNode[8] > OPEN[index, 8]:
                    continue
                else:
                    OPEN[index] = newNode
            else:  # if this node is not in the OPEN list
                OPEN[OPEN_COUNT] = newNode
                OPEN_CHECK[OPEN_COUNT] = 1
                OPEN_COUNT += 1
    
    # backward reconstruct path
    k = 0
    OptimalPath = np.zeros((200, 4))
    while not np.all(Root == currentNode):
        OptimalPath[k] = currentNode[:4]
        parentIndex = np.where(np.all(CLOSED[:, :4] == currentNode[4:8], axis=1))[0][0]
        currentNode = CLOSED[parentIndex]
        k += 1
    
    OptimalPath = OptimalPath[:k]
    OptimalPath = np.vstack((OptimalPath, Root[:4]))
    OptimalPath = np.flipud(OptimalPath)
    
    return OptimalPath.astype(int)


In [7]:
def MRPP_CBS(MapMat, StartStates, GoalStates, ctime):
    robotNum = StartStates.shape[0]
    PathCell = [None] * robotNum

    OPEN_COUNT = 1
    MAX_OPEN_SIZE = 1000
    MAX_TIME_STEP = 300
    OPEN = [None] * MAX_OPEN_SIZE
    OPEN_CHECK = np.zeros(MAX_OPEN_SIZE)

    # initialize
    class Node:
        pass

    Root = Node()
    Root.ID = 1
    Root.parent = 0
    Root.constraints = []
    solution = [None] * robotNum
    AllPath = [None] * robotNum
    costVec = np.zeros(robotNum)
    for i in range(robotNum):
        tempMat = MapMat.copy()
        tempMat[GoalStates[i, 0], GoalStates[i, 1]] = 0
        path = AStarSTDiff(tempMat, StartStates[i, :], GoalStates[i, :], [])
        path[:, 3] = path[:, 3] + ctime

        # make sure robots at goal positions will stay forever
        pathSize = path.shape[0]
        newPath = np.zeros((MAX_TIME_STEP, 4))
        newPath[:pathSize, :] = path
        newPath[pathSize:MAX_TIME_STEP, :3] = np.tile(path[-1, :3], (MAX_TIME_STEP - pathSize, 1))
        newPath[pathSize:MAX_TIME_STEP, 3] = np.arange(path[-1, 3] + 1, MAX_TIME_STEP)
        AllPath[i] = path
        solution[i] = newPath
        costVec[i] = path.shape[0] - 1
    Root.AllPath = AllPath
    Root.solution = solution
    Root.costVec = costVec
    Root.cost = np.sum(costVec)

    OPEN[0] = Root
    OPEN_CHECK[0] = 0

    while not np.all(OPEN_CHECK == 1):
        bestCost = 0
        bestNodeID = 0
        for i in range(OPEN_COUNT):
            if OPEN_CHECK[i] == 0:
                node = OPEN[i]
                if node.cost > bestCost:
                    bestNodeID = node.ID
                    bestCost = node.cost
        P = OPEN[bestNodeID]

        # conflict detection
        solution = P.solution
        conflictFoundFlag = 0
        constraints = np.zeros((2, 4))
        for i in range(robotNum):
            iPath = solution[i]
            iPath = iPath[:, :3]
            for j in range(robotNum):
                if j <= i:
                    continue

                jPath = solution[j]
                jPath = jPath[:, :3]
                iLen = iPath.shape[0]
                jLen = jPath.shape[0]
                if iLen > 1 and jLen > 1:
                    iMat = np.zeros((iLen - 1, 6))
                    iMat[:, :3] = iPath[:iLen - 1, :]
                    iMat[:, 3:] = iPath[1:iLen, :]
                    jMat = np.zeros((jLen - 1, 6))
                    jMat[:, 3:] = jPath[:jLen - 1, :]
                    jMat[:, :3] = jPath[1:jLen, :]
                temp = jMat[:, 2].copy()
                jMat[:, 2] = jMat[:, 5]
                jMat[:, 5] = temp

                # vertex and edge conflict detection
                vertexKind = np.isin(iPath, jPath)
                edgeKind = np.isin(iMat, jMat)
                vertex = np.where(np.all(vertexKind, axis=1))[0]
                edge = np.where(np.all(edgeKind, axis=1))[0]
                if vertex.size > 0 and edge.size > 0:  # both conflicts exist
                    vertexConflict = iPath[vertex[0]]
                    edgeConflict = iMat[edge[0]]
                    if vertexConflict[2] <= edgeConflict[5]:
                        constraints[0] = np.concatenate(([i], vertexConflict))
                        constraints[1] = np.concatenate(([j], vertexConflict))
                    else:
                        constraints[0] = np.concatenate(([i], edgeConflict[3:6]))
                        constraints[1] = np.concatenate(([j], edgeConflict[0:3], [edgeConflict[5]]))
                    conflictFoundFlag = 1
                    break
                elif vertex.size > 0 and edge.size == 0:  # only vertex conflict
                    conflictFoundFlag = 1
                    vertexConflict = iPath[vertex[0]]
                    constraints[0] = np.concatenate(([i], vertexConflict))
                    constraints[1] = np.concatenate(([j], vertexConflict))
                    break
                elif vertex.size == 0 and edge.size > 0:  # only edge conflict
                    conflictFoundFlag = 1
                    edgeConflict = iMat[edge[0]]
                    constraints[0] = np.concatenate(([i], edgeConflict[3:6]))
                    constraints[1] = np.concatenate(([j], edgeConflict[0:3], [edgeConflict[5]]))
                    break
                else:
                    continue
            if conflictFoundFlag == 1:
                break

        if conflictFoundFlag == 0:
            PathCell = P.AllPath
            break

        # branching and expanding the tree
        OPEN_CHECK[bestNodeID] = 1
        for i in range(2):
            agentID = int(constraints[i, 0])
            A = Node()
            A.ID = OPEN_COUNT + 1
            A.parent = P.ID
            A.constraints = np.concatenate((P.constraints, constraints[i:i+1, :]))
            pSolution = P.solution
            startRCA = StartStates[agentID]
            goalRCA = GoalStates[agentID]
            AConstraints = A.constraints
            AConstraints = AConstraints[AConstraints[:, 0] == agentID, 1:]
            tempMat = MapMat.copy()
            tempMat[goalRCA[0], goalRCA[1]] = 0
            path = AStarSTDiff(tempMat, startRCA, goalRCA, AConstraints)
            # make sure robots at goal positions will stay forever
            pathSize = path.shape[0]
            newPath = np.zeros((MAX_TIME_STEP, 4))
            newPath[:pathSize, :] = path
            newPath[pathSize:MAX_TIME_STEP, :3] = np.tile(path[-1, :3], (MAX_TIME_STEP - pathSize, 1))
            newPath[pathSize:MAX_TIME_STEP, 3] = np.arange(path[-1, 3] + 1, MAX_TIME_STEP)

            pSolution[agentID] = newPath
            A.solution = pSolution
            pSolution[agentID] = path
            A.AllPath = pSolution
            costVec = P.costVec.copy()
            costVec[agentID] = path.shape[0] - 1
            A.costVec = costVec
            A.cost = np.sum(costVec)

            OPEN[OPEN_COUNT] = A
            OPEN_CHECK[OPEN_COUNT] = 0
            OPEN_COUNT += 1

    return PathCell


In [8]:
def MRPP_CBSHWY(MapMat, Highway, StartStates, GoalStates, ctime):
    robotNum = StartStates.shape[0]
    PathCell = [None] * robotNum

    OPEN_COUNT = 1
    MAX_OPEN_SIZE = 1000
    MAX_TIME_STEP = 300
    OPEN = [None] * MAX_OPEN_SIZE
    OPEN_CHECK = np.zeros(MAX_OPEN_SIZE)

    # initialize
    class Node:
        pass

    Root = Node()
    Root.ID = 1
    Root.parent = 0
    Root.constraints = []
    solution = [None] * robotNum
    AllPath = [None] * robotNum
    costVec = np.zeros(robotNum)
    for i in range(robotNum):
        tempMat = MapMat.copy()
        tempMat[GoalStates[i, 0], GoalStates[i, 1]] = 0
        path = AStarSTDiffHWY(tempMat, Highway, StartStates[i, :], GoalStates[i, :], [])
        path[:, 3] = path[:, 3] + ctime

        # make sure robots at goal positions will stay forever
        pathSize = path.shape[0]
        newPath = np.zeros((MAX_TIME_STEP, 4))
        newPath[:pathSize, :] = path
        newPath[pathSize:MAX_TIME_STEP, :3] = np.tile(path[-1, :3], (MAX_TIME_STEP - pathSize, 1))
        newPath[pathSize:MAX_TIME_STEP, 3] = np.arange(path[-1, 3] + 1, MAX_TIME_STEP)
        AllPath[i] = path
        solution[i] = newPath
        costVec[i] = path.shape[0] - 1
    Root.AllPath = AllPath
    Root.solution = solution
    Root.costVec = costVec
    Root.cost = np.sum(costVec)

    OPEN[0] = Root
    OPEN_CHECK[0] = 0

    while not np.all(OPEN_CHECK == 1):
        bestCost = 0
        bestNodeID = 0
        for i in range(OPEN_COUNT):
            if OPEN_CHECK[i] == 0:
                node = OPEN[i]
                if node.cost > bestCost:
                    bestNodeID = node.ID
                    bestCost = node.cost
        P = OPEN[bestNodeID]

        # conflict detection
        solution = P.solution
        conflictFoundFlag = 0
        constraints = np.zeros((2, 4))
        for i in range(robotNum):
            iPath = solution[i]
            iPath = iPath[:, :3]
            for j in range(robotNum):
                if j <= i:
                    continue

                jPath = solution[j]
                jPath = jPath[:, :3]
                iLen = iPath.shape[0]
                jLen = jPath.shape[0]
                if iLen > 1 and jLen > 1:
                    iMat = np.zeros((iLen - 1, 6))
                    iMat[:, :3] = iPath[:iLen - 1, :]
                    iMat[:, 3:] = iPath[1:iLen, :]
                    jMat = np.zeros((jLen - 1, 6))
                    jMat[:, 3:] = jPath[:jLen - 1, :]
                    jMat[:, :3] = jPath[1:jLen, :]
                temp = jMat[:, 2].copy()
                jMat[:, 2] = jMat[:, 5]
                jMat[:, 5] = temp

                # vertex and edge conflict detection
                vertexKind = np.isin(iPath, jPath)
                edgeKind = np.isin(iMat, jMat)
                vertex = np.where(np.all(vertexKind, axis=1))[0]
                edge = np.where(np.all(edgeKind, axis=1))[0]
                if vertex.size > 0 and edge.size > 0:  # both conflicts exist
                    vertexConflict = iPath[vertex[0]]
                    edgeConflict = iMat[edge[0]]
                    if vertexConflict[2] <= edgeConflict[5]:
                        constraints[0] = np.concatenate(([i], vertexConflict))
                        constraints[1] = np.concatenate(([j], vertexConflict))
                    else:
                        constraints[0] = np.concatenate(([i], edgeConflict[3:6]))
                        constraints[1] = np.concatenate(([j], edgeConflict[0:3], [edgeConflict[5]]))
                    conflictFoundFlag = 1
                    break
                elif vertex.size > 0 and edge.size == 0:  # only vertex conflict
                    conflictFoundFlag = 1
                    vertexConflict = iPath[vertex[0]]
                    constraints[0] = np.concatenate(([i], vertexConflict))
                    constraints[1] = np.concatenate(([j], vertexConflict))
                    break
                elif vertex.size == 0 and edge.size > 0:  # only edge conflict
                    conflictFoundFlag = 1
                    edgeConflict = iMat[edge[0]]
                    constraints[0] = np.concatenate(([i], edgeConflict[3:6]))
                    constraints[1] = np.concatenate(([j], edgeConflict[0:3], [edgeConflict[5]]))
                    break
                else:
                    continue
            if conflictFoundFlag == 1:
                break

        if conflictFoundFlag == 0:
            PathCell = P.AllPath
            break

        # branching and expanding the tree
        OPEN_CHECK[bestNodeID] = 1
        for i in range(2):
            agentID = int(constraints[i, 0])
            A = Node()
            A.ID = OPEN_COUNT + 1
            A.parent = P.ID
            A.constraints = np.concatenate((P.constraints, constraints[i:i+1, :]))
            pSolution = P.solution
            startRCA = StartStates[agentID]
            goalRCA = GoalStates[agentID]
            AConstraints = A.constraints
            AConstraints = AConstraints[AConstraints[:, 0] == agentID, 1:]
            tempMat = MapMat.copy()
            tempMat[goalRCA[0], goalRCA[1]] = 0
            path = AStarSTDiffHWY(tempMat, Highway, startRCA, goalRCA, AConstraints)
            # make sure robots at goal positions will stay forever
            pathSize = path.shape[0]
            newPath = np.zeros((MAX_TIME_STEP, 4))
            newPath[:pathSize, :] = path
            newPath[pathSize:MAX_TIME_STEP, :3] = np.tile(path[-1, :3], (MAX_TIME_STEP - pathSize, 1))
            newPath[pathSize:MAX_TIME_STEP, 3] = np.arange(path[-1, 3] + 1, MAX_TIME_STEP - 1)

            pSolution[agentID] = newPath
            A.solution = pSolution
            pSolution[agentID] = path
            A.AllPath = pSolution
            costVec = P.costVec.copy()
            costVec[agentID] = path.shape[0] - 1
            A.costVec = costVec
            A.cost = np.sum(costVec)

            OPEN[OPEN_COUNT] = A
            OPEN_CHECK[OPEN_COUNT] = 0
            OPEN_COUNT += 1

    return PathCell


In [10]:
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

xlength = 61
ylength = 29
robotNum = 10
podNum = 800
depotNum = 8
taskNum = robotNum

def xy2rc(x, y):
    return np.array([ylength + 1 - y, x])

def rc2xy(r, c):
    return np.array([c, ylength + 1 - r])

def initialize(xlength, ylength, robotNum, podNum, depotNum):
    RobotStates = np.zeros((robotNum, 3))
    PodStates = np.zeros((podNum, 3))
    DepotStates = np.zeros((depotNum, 3))
    StorageOccupancy = np.zeros((ylength, xlength))
    return RobotStates, PodStates, DepotStates, StorageOccupancy

class Task:
    pass

TaskCell = []
for i in range(taskNum):
    task = Task()
    task.PodID = np.random.randint(1, podNum + 1)
    task.StationID = np.random.randint(1, depotNum + 1)
    task.ProcessTime = np.random.randint(5, 21)
    task.ReturnState = np.zeros(3)
    TaskCell.append(task)

np.save('taskSet.npy', TaskCell)
TaskCell = np.load('taskSet.npy', allow_pickle=True)

StartXYA = RobotStates
GoalXYA = np.zeros((robotNum, 3))
StartRCA = np.zeros((robotNum, 3))
GoalRCA = np.zeros((robotNum, 3))

temp = np.random.permutation(podNum)
for i in range(robotNum):
    StartRCA[i, :2] = xy2rc(StartXYA[i, 0], StartXYA[i, 1])
    StartRCA[i, 2] = StartXYA[i, 2]
    GoalXYA[i, :2] = PodStates[temp[i], :2]
    GoalRCA[i, :2] = xy2rc(GoalXYA[i, 0], GoalXYA[i, 1])
    GoalRCA[i, 2] = np.random.randint(1, 5)

AllPathCell = MRPP_CBS(MapOccupancy, StartRCA, GoalRCA, 0)

futureSize = 3
HeatMap = [np.zeros((robotNum, 3)) for _ in range(futureSize)]

fig, ax = plt.subplots()
ax.set_xlim(0, xlength + 1)
ax.set_ylim(0, ylength + 1)
ax.set_aspect('equal')
ax.set_xticks(np.arange(0, xlength + 1, 1))
ax.set_yticks(np.arange(0, ylength + 1, 1))
ax.grid(True)

def plotAll(AllRobotState, AllPodState, DepotStates):
    # Plotting code here
    pass

def update(frame):
    globalTime = frame + 1
    AllRobotState = np.zeros((robotNum, 3))
    AllPodState = PodStates.copy()
    
    for i in range(robotNum):
        if len(AllPathCell[i]) > 0:
            path = AllPathCell[i]
            if globalTime <= path.shape[0]:
                state = path[globalTime - 1]
                AllRobotState[i, :2] = rc2xy(state[0], state[1])
                AllRobotState[i, 2] = state[2]
            
            for j in range(futureSize):
                if globalTime + j <= path.shape[0]:
                    state = path[globalTime + j - 1]
                    HeatMap[j][i, :2] = rc2xy(state[0], state[1])
                    HeatMap[j][i, 2] = state[2]
                else:
                    state = path[-1]
                    HeatMap[j][i, :2] = rc2xy(state[0], state[1])
                    HeatMap[j][i, 2] = state[2]
    
    plotAll(AllRobotState, AllPodState, DepotStates)
    return []

ani = FuncAnimation(fig, update, frames=100, interval=500, repeat=False)
ani.save('simulation.mp4', writer='ffmpeg')

plt.show()


NameError: name 'RobotStates' is not defined

In [14]:
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

xlength = 61
ylength = 29
robotNum = 10
podNum = 800
depotNum = 8

xy2rc = lambda x, y: np.array([ylength + 1 - y, x])
rc2xy = lambda r, c: np.array([c, ylength + 1 - r])

sz = plt.get_current_fig_manager().window
sz.set_position((sz.x, 80))
sz.set_size((sz.width, 950))

fig, ax = plt.subplots()
ax.set_xlim(0, xlength + 1)
ax.set_ylim(0, ylength + 1)
ax.set_aspect('equal')
ax.set_xticks(np.arange(0, xlength + 1, 1))
ax.set_yticks(np.arange(0, ylength + 1, 1))
ax.grid(True)

globalTime = 1
GlobalRobotXYA = np.zeros((robotNum, 3))
GlobalPodXY = np.zeros((podNum, 2))

MapOccupancy = np.zeros((ylength, xlength))
RobotOccupancy = np.zeros((ylength, xlength))
PodOccupancy = np.zeros((ylength, xlength))
StorageOccupancy = np.zeros((ylength, xlength))

def initialize(xlength, ylength, robotNum, podNum, depotNum):
    RobotXYA = np.zeros((robotNum, 3))
    PodXY = np.zeros((podNum, 2))
    DepotXY = np.zeros((depotNum, 2))
    StorageOccupancy = np.zeros((ylength, xlength))
    return RobotXYA, PodXY, DepotXY, StorageOccupancy

def plotAll(RobotXYA, PodXY, DepotXY):
    # Plotting code here
    pass

RobotXYA, PodXY, DepotXY, StorageOccupancy = initialize(xlength, ylength, robotNum, podNum, depotNum)
PodOccupancy = StorageOccupancy

plotAll(RobotXYA, PodXY, DepotXY)

MapOccupancy = MapOccupancy + PodOccupancy
Highway = [None] * 2

mat = np.zeros((ylength, xlength))
mat[0, :] = np.ones(xlength)
mat[6, :] = 3 * np.ones(xlength)
mat[12, :] = np.ones(xlength)
mat[18, :] = 3 * np.ones(xlength)
mat[24, :] = np.ones(xlength)
mat[25, :] = 3 * np.ones(xlength)
mat[26, :] = np.ones(xlength)
mat[27, :] = 3 * np.ones(xlength)
mat[28, :] = np.ones(xlength)
Highway[0] = mat

mat = np.zeros((ylength, xlength))
mat[:, 0] = 2 * np.ones(ylength)
mat[:, 3] = 4 * np.ones(ylength)
mat[:, 6] = 2 * np.ones(ylength)
mat[:, 9] = 4 * np.ones(ylength)
mat[:, 12] = 2 * np.ones(ylength)
mat[:, 15] = 4 * np.ones(ylength)
mat[:, 18] = 2 * np.ones(ylength)
mat[:, 21] = 4 * np.ones(ylength)
mat[:, 24] = 2 * np.ones(ylength)
mat[:, 27] = 4 * np.ones(ylength)
mat[:, 30] = 2 * np.ones(ylength)
mat[:, 33] = 4 * np.ones(ylength)
mat[:, 36] = 2 * np.ones(ylength)
mat[:, 39] = 4 * np.ones(ylength)
mat[:, 42] = 2 * np.ones(ylength)
mat[:, 45] = 4 * np.ones(ylength)
mat[:, 48] = 2 * np.ones(ylength)
mat[:, 51] = 4 * np.ones(ylength)
mat[:, 54] = 2 * np.ones(ylength)
mat[:, 57] = 4 * np.ones(ylength)
mat[:, 60] = 2 * np.ones(ylength)
Highway[1] = mat

StartXYA = RobotXYA
GoalXYA = np.zeros((robotNum, 3))
StartRCA = np.zeros((robotNum, 3))
GoalRCA = np.zeros((robotNum, 3))

temp = np.random.permutation(podNum)
for i in range(robotNum):
    StartRCA[i, :2] = xy2rc(StartXYA[i, 0], StartXYA[i, 1])
    StartRCA[i, 2] = StartXYA[i, 2]
    GoalXYA[i, :2] = PodXY[temp[i], :]
    GoalRCA[i, :2] = xy2rc(GoalXYA[i, 0], GoalXYA[i, 1])
    GoalRCA[i, 2] = np.random.randint(1, 5)

AllPathCell = MRPP_CBSHWY(MapOccupancy, Highway, StartRCA, GoalRCA, 0)

futureSize = 3
HeatMap = [np.zeros((robotNum, 3)) for _ in range(futureSize)]

def update(frame):
    global globalTime
    frame += 1
    AllRobotState = np.zeros((robotNum, 3))
    AllPodState = PodXY.copy()

    for i in range(robotNum):
        if len(AllPathCell[i]) > 0:
            path = AllPathCell[i]
            if globalTime <= path.shape[0]:
                state = path[globalTime - 1]
                AllRobotState[i, :2] = rc2xy(state[0], state[1])
                AllRobotState[i, 2] = state[2]

            for j in range(futureSize):
                if globalTime + j <= path.shape[0]:
                    state = path[globalTime + j - 1]
                    HeatMap[j][i, :2] = rc2xy(state[0], state[1])
                    HeatMap[j][i, 2] = state[2]
                else:
                    state = path[-1]
                    HeatMap[j][i, :2] = rc2xy(state[0], state[1])
                    HeatMap[j][i, 2] = state[2]

    plotAll(AllRobotState, AllPodState, DepotXY)
    globalTime += 1
    return []

fig, ax = plt.subplots()
ax.set_xlim(0, xlength + 1)
ax.set_ylim(0, ylength + 1)
ax.set_aspect('equal')
ax.set_xticks(np.arange(0, xlength + 1, 1))
ax.set_yticks(np.arange(0, ylength + 1, 1))
ax.grid(True)

ani = FuncAnimation(fig, update, frames=100, interval=500, repeat=False)
ani.save('simulation.mp4', writer='ffmpeg')

plt.show()


AttributeError: 'FigureManagerBase' object has no attribute 'window'

<Figure size 640x480 with 0 Axes>

In [16]:
import numpy as np

xlength = 3
ylength = 3
MapMat = np.zeros((ylength, xlength))
MapMat[1, 1] = 1

Highway = []
temp = np.zeros((ylength, xlength))
temp[0, :] = np.ones(xlength)
temp[1, :] = 3 * np.ones(xlength)
temp[2, :] = np.ones(xlength)
Highway.append(temp)

temp = np.zeros((ylength, xlength))
temp[:, 0] = 2 * np.ones(ylength)
temp[:, 1] = 4 * np.ones(ylength)
temp[:, 2] = 4 * np.ones(ylength)
Highway.append(temp)

def AStarSTDiffHWY(MapMat, Highway, start, goal, constraints):
    # Implementation of A* algorithm
    # Your code here
    pass

path = AStarSTDiffHWY(MapMat, Highway, [1, 1, 1], [2, 3, 1], [])


In [21]:
import matplotlib.pyplot as plt

def plotAll(RobotStates, PodStates, DepotStates):
    xlength = 61
    ylength = 29
    
    plt.figure(figsize=(10, 10))
    plt.xlim(0, xlength + 1)
    plt.ylim(0, ylength + 1)
    plt.gca().set_aspect('equal')
    plt.xticks(range(xlength + 1))
    plt.yticks(range(ylength + 1))
    plt.grid(True, linewidth=5)
    
    plt.plot(DepotStates[:, 0], DepotStates[:, 1], 's', color='gray', markersize=30)
    plt.plot(PodStates[:, 0], PodStates[:, 1], 's', color='black', markersize=20)
    plt.plot(RobotStates[:, 0], RobotStates[:, 1], 'o', color='red', markersize=15)
    
    for i in range(RobotStates.shape[0]):
        temp = 0.3
        x = RobotStates[i, 0]
        y = RobotStates[i, 1]
        a = RobotStates[i, 2]
        
        if a == 1:
            xx = x + temp
            yy = y
        elif a == 2:
            xx = x
            yy = y + temp
        elif a == 3:
            xx = x - temp
            yy = y
        elif a == 4:
            xx = x
            yy = y - temp
        
        plt.plot([x, xx], [y, yy], color='black', linestyle='-', linewidth=4)
    
    plt.show()
    # Example usage
    RobotStates = np.array([[1, 2, 1], [3, 4, 2], [5, 6, 3]])
    PodStates = np.array([[7, 8], [9, 10], [11, 12]])
    DepotStates = np.array([[13, 14], [15, 16]])

    plotAll(RobotStates, PodStates, DepotStates)
